In [1]:
import os
import json
import labelbox as lb
import time

In [2]:
class UploadLabel(object):
    def __init__(self, api_key, project_id, dataset_name, mode_upload_image, images_directory_path=""):
        print("initialised")
        self.client = lb.Client(api_key=api_key)
        self.project = self.client.get_project(project_id)
        self.mode_upload_image = mode_upload_image
        self.images_directory_path = images_directory_path
        self.dataset_name = dataset_name
        
        
    def upload_batch_images(self, mode_upload_image):
        global_keys = os.listdir(self.images_directory_path) if mode_upload_image=="batch" else [os.path.basename(self.images_directory_path)]
        dataset = self.client.create_dataset(name=self.dataset_name)
        assets = [
            {
                "row_data": os.path.join(self.images_directory_path, global_key) if mode_upload_image=="batch" else self.images_directory_path, 
                "global_key": global_key,
                "media_type": "VIDEO"
            } for global_key in global_keys
        ]
        task = dataset.create_data_rows(assets)
        task.wait_till_done()
        
        if task.errors:
            print("Errors :", task.errors)
            return  # Exit early if there are errors
    
        print("Failed data rows:", task.failed_data_rows)
        
        batch = self.project.create_batch(
            self.dataset_name+"_"+str(int(time.time())), 
            global_keys=global_keys, 
            priority=5 
        )
        print(batch, self.dataset_name+"_"+str(int(time.time())))

              
    def run(self):
        if self.mode_upload_image in ["single", "batch"]:
            self.upload_batch_images(self.mode_upload_image)

In [ ]:
if __name__ == '__main__':
    api_key = "xxx"
    project_id = "xxx"
    mode_upload_image = 'batch'
    images_directory_path = r'xxx'
    dataset_name = 'xxx'

    u = UploadLabel(api_key, project_id, dataset_name, mode_upload_image, images_directory_path)
    u.run()